# Data Processing for the Sankey Diagram

This jupyter notebook contains the data processing steps for realizing the transfer Sankey diagram on our website. 

The Sankey diagram will show the transfers between different leagues at different career stages of the players. The data set we have at hand is the fifa game dataset that contains the player_id, fifa_version, name, overall rating and club name, league name for each player. In the end, we need to extract the transfer route of each player, and discover whether there is any observable patterns that can be visualized using a Sankey diagram. 

In the end, a Json object is generated to contain the nodes (and the connections between the nodes), that can be used in a javascript. 

Link to the dataset: https://www.datacamp.com/tutorial/markdown-in-jupyter-notebook


In [1]:
import pandas as pd
import numpy as np

# Loading in the data.

In [2]:
# Loading in the data
male_players_df = pd.read_csv('./data/male_players.csv', usecols=['player_id', 'fifa_version', 'short_name', 'overall', 'age', 'club_name', 'league_name'])

In [3]:
# Print out the top 10 players for a sanity check
display(male_players_df.head(10))
print(male_players_df.shape)

,player_id,fifa_version,short_name,overall,age,league_name,club_name
0,158023,23,L. Messi,91,35,Ligue 1,Paris Saint Germain
1,165153,23,K. Benzema,91,34,La Liga,Real Madrid
2,188545,23,R. Lewandowski,91,33,La Liga,FC Barcelona
3,192985,23,K. De Bruyne,91,31,Premier League,Manchester City
4,231747,23,K. Mbappé,91,23,Ligue 1,Paris Saint Germain
5,192119,23,T. Courtois,90,30,La Liga,Real Madrid
6,209331,23,M. Salah,90,30,Premier League,Liverpool
7,167495,23,M. Neuer,89,36,Bundesliga,FC Bayern München
8,190871,23,Neymar Jr,89,30,Ligue 1,Paris Saint Germain
9,200145,23,Casemiro,89,30,Premier League,Manchester United


(10003590, 7)


# Preprocessing of the Data

Remove duplicates of the same player. 

In [4]:
# Set pandas display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Drop duplicates based on 'player_id' and 'fifa_version'
players = male_players_df.drop_duplicates(subset=['player_id', 'fifa_version'])

print(players[players['player_id'] == 158023])

         player_id  fifa_version short_name  overall  age league_name  \
0           158023            23   L. Messi       91   35     Ligue 1   
166674      158023            22   L. Messi       92   34     Ligue 1   
1385125     158023            21   L. Messi       93   33     Ligue 1   
2582753     158023            20   L. Messi       94   32     La Liga   
3711090     158023            19   L. Messi       94   31     La Liga   
4941076     158023            18   L. Messi       94   30     La Liga   
6453436     158023            17   L. Messi       93   29     La Liga   
8054866     158023            16   L. Messi       94   28     La Liga   
9041041     158023            15   L. Messi       93   27     La Liga   

                   club_name  
0        Paris Saint Germain  
166674   Paris Saint Germain  
1385125  Paris Saint Germain  
2582753         FC Barcelona  
3711090         FC Barcelona  
4941076         FC Barcelona  
6453436         FC Barcelona  
8054866         FC Ba

Filtering the players, so only the players who had been present all the time (from FIFA 15 to FIFA 23) are kept. And also they need to be younger than 25 in FIFA 15, so we are indeed tracking some young player advancing through their career.  

In [5]:
# Group by player_id and count the number of unique fifa_versions for each player
player_counts = players.groupby('player_id')['fifa_version'].nunique()

# Filter out players who have entries for all fifa versions (15 to 23)
players_all_fifas = player_counts[player_counts == 9].index

# Filter the players DataFrame to keep only players who have entries for all fifa versions
players_all9 = players[players['player_id'].isin(players_all_fifas)]

print(players_all9.shape[0]/9)

3728.0


In [6]:
# Filter the DataFrame to keep only players who were younger than 25 in FIFA 15
players_under25_in_fifa15 = players_all9[(players_all9['fifa_version'] == 15) & (players_all9['age'] < 25) & (players_all9['age'] >22)]

# Filter the original DataFrame to keep entries for players who were younger than 25 in FIFA 15 across all FIFA versions
players_filtered = players_all9[players_all9['player_id'].isin(players_under25_in_fifa15['player_id'])]

print(players_filtered.head(10))
# print(players_filtered[players_filtered['player_id'] == 183277])

print("\nNumber of players: ")
print(players_filtered.shape[0]/9)




    player_id  fifa_version    short_name  overall  age     league_name  \
3      192985            23  K. De Bruyne       91   31  Premier League   
20     182521            23      T. Kroos       88   32         La Liga   
26     215914            23      N. Kanté       88   31  Premier League   
30     193080            23        De Gea       87   31  Premier League   
41     186153            23   W. Szczęsny       86   32         Serie A   
42     188943            23      K. Trapp       86   31      Bundesliga   
43     189509            23        Thiago       86   31  Premier League   
45     189596            23     T. Müller       86   32      Bundesliga   
46     192387            23   C. Immobile       86   32         Serie A   
48     201024            23  K. Koulibaly       86   31  Premier League   

              club_name  
3       Manchester City  
20          Real Madrid  
26              Chelsea  
30    Manchester United  
41             Juventus  
42  Eintracht Fran

# Tracking the Transfer Route

Take the 500 best players and start tracking them from 2015 to 2023. 

In [7]:
from collections import Counter

# Defining a function to find the mode
def mode(array):
    # Convert all elements to strings
    array = np.array(array, dtype=str)
    # Flatten the array
    flattened_data = array.flatten()
    # Count occurrences of each string
    string_counts = Counter(flattened_data)
    # Find the most common string
    most_common_string = max(string_counts, key=string_counts.get)
    return most_common_string


In [8]:
# This is 500 hundred players, giving you the player ID
players_500 = players_filtered.drop_duplicates(subset=['player_id']).head(1000)

# So it is a 500x1 numpy array
ID = players_500['player_id'].to_numpy().reshape(-1, 1)

# The array we are going to sort later
leagues = np.zeros((500, 3))

i = 0
# Going through all the players
for player_id in ID:

    # Getting this particular player
    id = player_id[0]
    player = players_filtered[players_filtered['player_id'] == id]

    
    early = player[player['age'] < 25].league_name.to_numpy().reshape(-1, 1)
    mid = player[(25 <= player['age']) & (player['age'] <= 30)].league_name.to_numpy().reshape(-1, 1)
    late = player[player['age'] > 30].league_name.mode

In [9]:
# This is 500 hundred players, giving you the player ID
players_500 = players_filtered.drop_duplicates(subset=['player_id']).head(500)

# So it is a 500x1 numpy array
ID = players_500['player_id'].to_numpy().reshape(-1, 1)

# The array we are going to sort later
leagues = np.zeros((500, 3), dtype='object') 

i = 0
# Going through all the players
for player_id in ID:

    # Getting this particular player
    id = player_id[0]
    player = players_filtered[players_filtered['player_id'] == id]

    just_league = player.league_name.to_numpy().reshape(-1, 1)
    
    early = just_league[-1][0]
    mid = mode(just_league[1:-1])
    late = just_league[0][0]

    leagues[i, 0] = str(early) + "(early)"
    leagues[i, 1] = str(mid) + "(mid)"
    leagues[i, 2] = str(late) + "(late)"

    i = i+1

Printing out the players and their leagues for a sanity check

In [10]:
print(players_500)

      player_id  fifa_version              short_name  overall  age  \
3        192985            23            K. De Bruyne       91   31   
20       182521            23                T. Kroos       88   32   
26       215914            23                N. Kanté       88   31   
30       193080            23                  De Gea       87   31   
41       186153            23             W. Szczęsny       86   32   
42       188943            23                K. Trapp       86   31   
43       189509            23                  Thiago       86   31   
45       189596            23               T. Müller       86   32   
46       192387            23             C. Immobile       86   32   
48       201024            23            K. Koulibaly       86   31   
63       185122            23              P. Gulácsi       85   32   
64       186942            23             İ. Gündoğan       85   31   
66       188377            23               K. Walker       85   32   
76    

In [11]:
print(leagues)

[['Bundesliga(early)' 'Premier League(mid)' 'Premier League(late)']
 ['La Liga(early)' 'La Liga(mid)' 'La Liga(late)']
 ['Ligue 1(early)' 'Premier League(mid)' 'Premier League(late)']
 ...
 ['Ekstraklasa(early)' 'Ekstraklasa(mid)' 'Ekstraklasa(late)']
 ['Bundesliga(early)' 'Ekstraklasa(mid)' 'Ekstraklasa(late)']
 ['Ekstraklasa(early)' 'Ekstraklasa(mid)' 'Ekstraklasa(late)']]


# Finding the Pattern

Sort the league array in a alphabetical order so we can count the number of occurance of each unique transfer route combination later

In [12]:
np.set_printoptions(threshold=np.inf)

def sort_rows_alphabetically(array):
    # Convert the array elements to strings
    array = np.array(array, dtype=str)
    # Remove rows containing any non-string elements
    array = array[np.all(array != 'nan', axis=1)]
    if array.size == 0:
        return array  # Return empty array if all rows contain 'nan'
    # Get the indices that would sort the array by the first column
    sorted_indices = np.lexsort(array.T[::-1])
    # Use the sorted indices to reorder the array
    sorted_array = array[sorted_indices]
    return sorted_array


sorted = sort_rows_alphabetically(leagues)
print("Sorted data:")
print(sorted)

Sorted data:
[['2. Bundesliga(early)' '2. Bundesliga(mid)' '2. Bundesliga(late)']
 ['2. Bundesliga(early)' '2. Bundesliga(mid)' '2. Bundesliga(late)']
 ['2. Bundesliga(early)' '2. Bundesliga(mid)' '2. Bundesliga(late)']
 ['2. Bundesliga(early)' '2. Bundesliga(mid)' '2. Bundesliga(late)']
 ['2. Bundesliga(early)' '2. Bundesliga(mid)' '2. Bundesliga(late)']
 ['2. Bundesliga(early)' '2. Bundesliga(mid)' '3. Liga(late)']
 ['2. Bundesliga(early)' '2. Bundesliga(mid)' 'Bundesliga(late)']
 ['2. Bundesliga(early)' '2. Bundesliga(mid)' 'Jupiler Pro League(late)']
 ['2. Bundesliga(early)' '3. Liga(mid)' '2. Bundesliga(late)']
 ['2. Bundesliga(early)' 'Bundesliga(mid)' '1. Division(late)']
 ['2. Bundesliga(early)' 'Bundesliga(mid)' 'A-League(late)']
 ['2. Bundesliga(early)' 'Bundesliga(mid)' 'Bundesliga(late)']
 ['2. Bundesliga(early)' 'Bundesliga(mid)' 'Bundesliga(late)']
 ['2. Bundesliga(early)' 'Bundesliga(mid)' 'Bundesliga(late)']
 ['2. Bundesliga(early)' 'Bundesliga(mid)' 'Bundesliga(late)']

Counting the number of occurance for each unique combination of trans route

In [13]:
from collections import Counter

def filter_and_replace(array):
    # Get the number of columns in the array
    num_columns = array.shape[1]
    
    # Dictionary to store the top 10 entries for each column
    top_10_entries = {}
    
    # Loop through each column
    for col_index in range(num_columns):
        # Get the column
        column = array[:, col_index]
        
        # Count the occurrences of each entry in the column
        counts = Counter(column)
        
        # Get the top 10 most common entries
        top_10 = counts.most_common(10)
        
        # Store the top 10 entries for the column
        top_10_entries[col_index] = [entry for entry, _ in top_10]
    
    # Filter and replace entries
    for row_index in range(array.shape[0]):
        for col_index in range(num_columns):
            entry = array[row_index, col_index]
            if entry not in top_10_entries[col_index]:
                if col_index == 0: 
                    array[row_index, col_index] = 'others(early)'
                if col_index == 1: 
                    array[row_index, col_index] = 'others(mid)'
                if col_index == 2: 
                    array[row_index, col_index] = 'others(late)'
    
    return array

filtered_data = filter_and_replace(sorted)
print(filtered_data)

[['2. Bundesliga(early)' 'others(mid)' 'others(late)']
 ['2. Bundesliga(early)' 'others(mid)' 'others(late)']
 ['2. Bundesliga(early)' 'others(mid)' 'others(late)']
 ['2. Bundesliga(early)' 'others(mid)' 'others(late)']
 ['2. Bundesliga(early)' 'others(mid)' 'others(late)']
 ['2. Bundesliga(early)' 'others(mid)' 'others(late)']
 ['2. Bundesliga(early)' 'others(mid)' 'Bundesliga(late)']
 ['2. Bundesliga(early)' 'others(mid)' 'others(late)']
 ['2. Bundesliga(early)' 'others(mid)' 'others(late)']
 ['2. Bundesliga(early)' 'Bundesliga(mid)' 'others(late)']
 ['2. Bundesliga(early)' 'Bundesliga(mid)' 'others(late)']
 ['2. Bundesliga(early)' 'Bundesliga(mid)' 'Bundesliga(late)']
 ['2. Bundesliga(early)' 'Bundesliga(mid)' 'Bundesliga(late)']
 ['2. Bundesliga(early)' 'Bundesliga(mid)' 'Bundesliga(late)']
 ['2. Bundesliga(early)' 'Bundesliga(mid)' 'Bundesliga(late)']
 ['2. Bundesliga(early)' 'Bundesliga(mid)' 'Bundesliga(late)']
 ['2. Bundesliga(early)' 'Bundesliga(mid)' 'Bundesliga(late)']
 ['2.

# Converting to JSON

Convert the format to JSON, so later we can copy-pasting it to a html. 

In [14]:
import json

def convert_to_sankey_json(data):
    # Convert data to list of lists with native Python data types
    data_list = data.tolist()
    
    unique_values, counts = np.unique(data, return_counts=True)
    nodes = [{"name": str(value)} for value in unique_values]
    links = []

    for row in data_list:
        for i in range(len(row) - 1):
            source_index = int(np.where(unique_values == row[i])[0][0])
            target_index = int(np.where(unique_values == row[i+1])[0][0])
            # Check if the link already exists
            existing_link = next((link for link in links if link["source"] == source_index and link["target"] == target_index), None)
            if existing_link:
                existing_link["value"] += 1
            else:
                links.append({"source": source_index, "target": target_index, "value": 1})

    return {"nodes": nodes, "links": links}

# Convert data to Sankey JSON format
sankey_json = convert_to_sankey_json(filtered_data)

# Convert dictionary to JSON string with manual line breaks
sankey_json_str = json.dumps(sankey_json, indent=2)
sankey_json_str = sankey_json_str.replace('},', '},\n')
print(sankey_json_str)

{
  "nodes": [
    {
      "name": "2. Bundesliga(early)"
    },

    {
      "name": "Bundesliga(early)"
    },

    {
      "name": "Bundesliga(late)"
    },

    {
      "name": "Bundesliga(mid)"
    },

    {
      "name": "Championship(early)"
    },

    {
      "name": "Championship(late)"
    },

    {
      "name": "Championship(mid)"
    },

    {
      "name": "La Liga 2(early)"
    },

    {
      "name": "La Liga 2(late)"
    },

    {
      "name": "La Liga 2(mid)"
    },

    {
      "name": "La Liga(early)"
    },

    {
      "name": "La Liga(late)"
    },

    {
      "name": "La Liga(mid)"
    },

    {
      "name": "Liga Portugal(early)"
    },

    {
      "name": "Liga Profesional(early)"
    },

    {
      "name": "Liga Profesional(late)"
    },

    {
      "name": "Liga Profesional(mid)"
    },

    {
      "name": "Ligue 1(early)"
    },

    {
      "name": "Ligue 1(late)"
    },

    {
      "name": "Ligue 1(mid)"
    },

    {
      "name": "Major League 